Étape 1 : Charger les données

Je vérifiecohérence entre les colonnes de train.csv et test.csv, car cela peut effectivement poser problème lors de la prédiction:
- train.csv a une colonne Survived (car c’est ce que tu veux apprendre à prédire).
- test.csv n’a pas cette colonne (car c’est ce que tu veux prédire).

Donc l'absence de Survived dans test.csv est parfaitement normale.

In [ ]:
# Étape 1 : Charger les données
import pandas as pd

# Chargement des données
train_df = pd.read_csv('data/train.csv')

# Aperçu des données
print(" Données d'entraînement (train.csv) :")
print(train_df.head())
print(f"Forme du train.csv : {train_df.shape}")



 Données d'entraînement (train.csv) :
---------------------------
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123 

Étape 2 — Inspection des données

In [256]:
""" Détecter les doublons """

import pandas as pd


#  Détecter les lignes dupliquées
doublons = train_df.duplicated()

#  Compter le nombre total de doublons
nombre_doublons = doublons.sum()

print(f" Nombre de lignes dupliquées dans train.csv : {nombre_doublons}")


 Nombre de lignes dupliquées dans train.csv : 0


In [257]:
""" Détecter les valeurs manquanntes """

import pandas as pd

# Calcul du nombre et du pourcentage de valeurs manquantes
missing_count = train_df.isnull().sum()
missing_percent = 100 * missing_count / len(train_df)

# Rassembler dans un DataFrame lisible
missing_data = pd.DataFrame({
    'Valeurs manquantes': missing_count,
    '% du total': missing_percent
})

# Filtrer les colonnes avec au moins 1 valeur manquante
missing_data = missing_data[missing_data['Valeurs manquantes'] > 0]

# Trier par pourcentage décroissant
missing_data = missing_data.sort_values(by='% du total', ascending=False)

# Afficher les résultats
print("📊 Valeurs manquantes dans train.csv :")
print(missing_data)


📊 Valeurs manquantes dans train.csv :
          Valeurs manquantes  % du total
Cabin                    687   77.104377
Age                      177   19.865320
Embarked                   2    0.224467


Étape 3 - Traitement des données manquantes

Étape 3-1 — Extraire le Deck : 

Chaque cabine a une valeur comme "C85" ou "E46" ou "C23 C25 C27".
On veut extraire uniquement la première lettre, qui indique le pont du navire.

In [258]:
# Extraire la première lettre (deck)
train_df["Deck"] = train_df["Cabin"].str.extract(r'([A-Za-z])')
print(train_df["Deck"])

0      NaN
1        C
2      NaN
3        C
4      NaN
      ... 
886    NaN
887      B
888    NaN
889      C
890    NaN
Name: Deck, Length: 891, dtype: object


Étape 3-2 — Créer Has_Cabin (0 ou 1)
On veut savoir si une personne avait une cabine renseignée ou non.
Cela peut refléter un statut social élevé ou un accès plus facile aux canots de sauvetage.

In [259]:
# Créer une colonne binaire : 1 si la cabine est connue, 0 sinon
train_df["Has_Cabin"] = train_df["Cabin"].notnull().astype(int)
print(train_df[["Has_Cabin","Name"]])

     Has_Cabin                                               Name
0            0                            Braund, Mr. Owen Harris
1            1  Cumings, Mrs. John Bradley (Florence Briggs Th...
2            0                             Heikkinen, Miss. Laina
3            1       Futrelle, Mrs. Jacques Heath (Lily May Peel)
4            0                           Allen, Mr. William Henry
..         ...                                                ...
886          0                              Montvila, Rev. Juozas
887          1                       Graham, Miss. Margaret Edith
888          0           Johnston, Miss. Catherine Helen "Carrie"
889          1                              Behr, Mr. Karl Howell
890          0                                Dooley, Mr. Patrick

[891 rows x 2 columns]


In [260]:
# Vérification
print(train_df["Deck"].value_counts(dropna=False))  # Y compris les NaN
print(train_df["Has_Cabin"].value_counts())

Deck
NaN    687
C       59
B       47
D       33
E       32
A       15
F       13
G        4
T        1
Name: count, dtype: int64
Has_Cabin
0    687
1    204
Name: count, dtype: int64


Étape 3-3 — Vérifier les valeurs manquantes sur la colonne Age

In [261]:
train_df["Age"].isnull().sum()


np.int64(177)

Étape 3-4 — Imputation avancée Age par combinaison Title, Sex, Pclass

In [262]:
# Extraire les titres depuis Name
train_df["Title"] = train_df["Name"].str.extract(r',\s*([^\.]+)\.')
print(train_df["Title"].value_counts())

Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
Ms                1
Mme               1
Don               1
Lady              1
Sir               1
Capt              1
the Countess      1
Jonkheer          1
Name: count, dtype: int64


In [263]:
#  Nettoyer et regrouper les titres rares
title_mapping = {
    "Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs",
    "Lady": "Rare", "Countess": "Rare", "Capt": "Rare",
    "Col": "Rare", "Don": "Rare", "Dr": "Rare", "Major": "Rare",
    "Rev": "Rare", "Sir": "Rare", "Jonkheer": "Rare", "Dona": "Rare", "the Countess": "Rare", 
}
train_df["Title"] = train_df["Title"].replace(title_mapping)

print(train_df["Title"].value_counts())


Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64


Étape 3-4 — Calculer l'âge moyen pour chaque combinaison Title + Sex + Pclass

In [264]:
# On crée un tableau à 3 dimensions :
# Calcul des moyennes d'âge
age_means = train_df.groupby(['Title', 'Sex', 'Pclass'])['Age'].mean()
print(age_means)
# age_means contient maintenant les valeurs moyennes d’âge pour chaque combinaison.

Title   Sex     Pclass
Master  male    1          5.306667
                2          2.258889
                3          5.350833
Miss    female  1         29.744681
                2         22.560606
                3         16.123188
Mr      male    1         41.580460
                2         32.768293
                3         28.724891
Mrs     female  1         40.400000
                2         33.682927
                3         33.515152
Rare    female  1         43.333333
        male    1         48.727273
                2         42.000000
Name: Age, dtype: float64


In [265]:
# fonction d’imputation selon ces 3 critères
def impute_age_advanced(row):
    if pd.isnull(row['Age']):
        return age_means.loc[row['Title'], row['Sex'], row['Pclass']]
    else:
        return row['Age']

train_df["Age"] = train_df.apply(impute_age_advanced, axis=1)


In [266]:
# Vérification
print(train_df["Age"].isnull().sum())  # Doit afficher 0


0


In [267]:
# Exporter les données avec implentamentio de l'Age
train_df.to_csv("train_implen_age.csv", index=False)

Étape 4 — Encoder les variables catégorielles
Transformer les colonnes comme Sex, Embarked, Title, Deck en nombres pour qu’elles soient compatibles avec le modèle.
On va faire dans cet ordre :

Sex → label encoding (0/1)

Embarked → one-hot encoding (S, C, Q)

Title → one-hot encoding (Mr, Miss, etc.)

Deck → one-hot encoding (A, B, ..., NaN)

Supprimer les colonnes inutiles (Name, Ticket, Cabin, etc.)

In [268]:
# 1. Sex
train_df["Sex"] = train_df["Sex"].map({"male": 0, "female": 1})

# 2. Embarked
train_df["Embarked"].fillna("S", inplace=True)
embarked_dummies = pd.get_dummies(train_df["Embarked"], prefix="Embarked")
train_df = pd.concat([train_df, embarked_dummies], axis=1)
train_df.drop("Embarked", axis=1, inplace=True)

# 3. Title
title_dummies = pd.get_dummies(train_df["Title"], prefix="Title")
train_df = pd.concat([train_df, title_dummies], axis=1)
train_df.drop("Title", axis=1, inplace=True)

# 4. Deck
deck_dummies = pd.get_dummies(train_df["Deck"], prefix="Deck")
train_df = pd.concat([train_df, deck_dummies], axis=1)
train_df.drop("Deck", axis=1, inplace=True)

# 5. Supprimer les colonnes non nécessaires
train_df.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"], inplace=True, errors="ignore")


/tmp/ipykernel_3526/2383733863.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["Embarked"].fillna("S", inplace=True)


In [269]:
print(train_df.dtypes)


Survived          int64
Pclass            int64
Sex               int64
Age             float64
SibSp             int64
Parch             int64
Fare            float64
Has_Cabin         int64
Embarked_C         bool
Embarked_Q         bool
Embarked_S         bool
Title_Master       bool
Title_Miss         bool
Title_Mr           bool
Title_Mrs          bool
Title_Rare         bool
Deck_A             bool
Deck_B             bool
Deck_C             bool
Deck_D             bool
Deck_E             bool
Deck_F             bool
Deck_G             bool
Deck_T             bool
dtype: object


In [270]:
# Convertir les colonnes bool → int (Gradient Boosting les traite mieux comme 0/1)

bool_cols = train_df.select_dtypes('bool').columns
train_df[bool_cols] = train_df[bool_cols].astype(int)

In [271]:
# Vérification
print(train_df.dtypes)


Survived          int64
Pclass            int64
Sex               int64
Age             float64
SibSp             int64
Parch             int64
Fare            float64
Has_Cabin         int64
Embarked_C        int64
Embarked_Q        int64
Embarked_S        int64
Title_Master      int64
Title_Miss        int64
Title_Mr          int64
Title_Mrs         int64
Title_Rare        int64
Deck_A            int64
Deck_B            int64
Deck_C            int64
Deck_D            int64
Deck_E            int64
Deck_F            int64
Deck_G            int64
Deck_T            int64
dtype: object


### Étape 5 — Entraînement du modèle avec Gradient Boosting avec les données d'entrainement


In [272]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# Chargement des données (tu peux adapter le chemin si besoin)

# Séparation des features et de la cible
X = train_df.drop("Survived", axis=1)
y = train_df['Survived']


# Train / validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation et entraînement du modèle
gb_model = GradientBoostingClassifier(
    n_estimators=100,       # nombre d'arbres
    learning_rate=0.1,      # taux d'apprentissage
    max_depth=3,            # profondeur max des arbres
    random_state=42
)

gb_model.fit(X_train, y_train)

# Évaluation sur validation
y_pred = gb_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# (Optionnel) Validation croisée
scores = cross_val_score(gb_model, X, y, cv=5)
print(f"Cross-validation mean accuracy: {scores.mean():.4f}")


Accuracy: 0.8101
Cross-validation mean accuracy: 0.8238


Etape 5-1 - Optimisation des hyperparamètres d'entrainement Gradien Boosting avec GridSearchCV

In [273]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# 1. Définir le modèle de base
gb = GradientBoostingClassifier(random_state=42)

# 2. Grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4],
    'min_samples_split': [2, 5],
    'subsample': [0.8, 1.0]
}

# 3. Configuration de GridSearchCV
grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    cv=5,                  # validation croisée à 5 plis
    scoring='accuracy',    # on cherche la meilleure précision
    n_jobs=-1,             # utilise tous les cœurs CPU
    verbose=2              # affiche la progression
)

# 4. Lancer la recherche
grid_search.fit(X, y)

# 5. Résultats
print(" Meilleurs paramètres :", grid_search.best_params_)
print(f" Meilleure précision moyenne : {grid_search.best_score_:.4f}")

# 6. Récupérer le meilleur modèle
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=3, min_samples_split=2, n_estimators=100,

Étape 5-2 — Entraînement du modèle avec Gradient Boosting et optimisation des hyperparameters

 Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 200, 'subsample': 0.8}


In [274]:
# Initialisation et entraînement du modèle avec les nouveaux hyperparameter
gb_model_optimised = GradientBoostingClassifier(
    n_estimators=200,       # nombre d'arbres
    learning_rate=0.1,      # taux d'apprentissage
    max_depth=3,            # profondeur max des arbres
    random_state=42,
    min_samples_split= 5,
    subsample=0.8,
)

gb_model_optimised.fit(X_train, y_train)

# Évaluation sur validation
y_pred = gb_model_optimised.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# (Optionnel) Validation croisée
scores = cross_val_score(gb_model_optimised, X, y, cv=5)
print(f"Cross-validation mean accuracy: {scores.mean():.4f}")


Accuracy: 0.8268
Cross-validation mean accuracy: 0.8440


## Étape 6 — Entraînement du modèle avec Gradient Boosting avec les données de test
